In [66]:
import pandas as pd
import numpy as np

In [36]:
results_major = pd.read_csv(r'C:\Users\evani\OneDrive\AI leiden\Sanquin\Github Merel\RL_matching-main\results\request_scratch\a0.1_g0.5_b50_0\request_9.csv')
results_C = pd.read_csv(r"C:\Users\evani\OneDrive\AI leiden\Sanquin\Github Merel\RL_matching-main\results\request_scratch\a0.1_g0.5_b50_['C']\request_9.csv")
results_Cc = pd.read_csv(r"C:\Users\evani\OneDrive\AI leiden\Sanquin\Github Merel\RL_matching-main\results\request_scratch\a0.1_g0.5_b50_['C', 'c']\request_9.csv")
results_CcE = pd.read_csv(r"C:\Users\evani\OneDrive\AI leiden\Sanquin\Github Merel\RL_matching-main\results\request_scratch\a0.1_g0.5_b50_['C', 'c', 'E']\request_9.csv")
results_CcEe = pd.read_csv(r"C:\Users\evani\OneDrive\AI leiden\Sanquin\Github Merel\RL_matching-main\results\request_scratch\a0.1_g0.5_b50_['C', 'c', 'E', 'e']\request_9.csv")
results_CcEeK = pd.read_csv(r"C:\Users\evani\OneDrive\AI leiden\Sanquin\Github Merel\RL_matching-main\results\request_scratch\a0.1_g0.5_b50_['C', 'c', 'E', 'e', 'K']\request_9.csv")

In [49]:
results_C

,day,logged,location,model name,supply scenario,demand scenario,avg daily demand,inventory size,test days,init days,...,num AB+ in inventory,reward,issued but nonexistent,issued but discarded,num outdates,num shortages,num mismatches A,num mismatches B,num mismatches D,num mismatches C
0,0,True,reg_9,request_scratch,cau100_afr0_asi0_9,regional_9,50,150,365,70,...,0,0.0,0,0,0.0,0,0,0,0,0
1,1,True,reg_9,request_scratch,cau100_afr0_asi0_9,regional_9,50,150,365,70,...,0,0.0,0,0,0.0,0,0,0,0,0
2,2,True,reg_9,request_scratch,cau100_afr0_asi0_9,regional_9,50,150,365,70,...,0,-60.0,1,0,0.0,1,0,0,0,0
3,3,True,reg_9,request_scratch,cau100_afr0_asi0_9,regional_9,50,150,365,70,...,0,-720.0,12,0,0.0,12,0,0,0,0
4,4,True,reg_9,request_scratch,cau100_afr0_asi0_9,regional_9,50,150,365,70,...,0,-660.0,11,0,0.0,11,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,430,True,reg_9,request_scratch,cau100_afr0_asi0_9,regional_9,50,150,365,70,...,0,-2706.0,41,0,246.0,41,0,0,0,0
431,431,True,reg_9,request_scratch,cau100_afr0_asi0_9,regional_9,50,150,365,70,...,0,-1800.0,30,0,0.0,30,0,0,0,0
432,432,True,reg_9,request_scratch,cau100_afr0_asi0_9,regional_9,50,150,365,70,...,0,-1266.0,20,0,66.0,20,0,0,0,0
433,433,True,reg_9,request_scratch,cau100_afr0_asi0_9,regional_9,50,150,365,70,...,0,-1260.0,21,0,0.0,21,0,0,0,0


In [1]:
dic = {'a:':1, 'b':2, 'c':3
       }

In [5]:
dic['a:']

1